In [ ]:
!pip3 uninstall torch -y
!pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu118 --force

In [4]:
import torch
from torch._dynamo import optimize
from typing import *
from torch import _dynamo

All the valid backends

In [5]:
_dynamo.list_backends()

['aot_ts_nvfuser',
 'cudagraphs',
 'inductor',
 'ipex',
 'nvprims_nvfuser',
 'onnxrt',
 'tensorrt',
 'tvm']

# optimizer()

A naive example

In [37]:
from torch import nn
import torch.nn.functional as F


class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc = nn.Linear(128, 10)

  def forward(self, x):
    x = self.fc(x)
    x = F.relu(x)
    return x

In [38]:
from torch._inductor import config as inductor_config
from torch._dynamo import config as dynamo_config

inductor_config.debug = True
dynamo_config.verbose = True

In [47]:
foo = Net()
foo = torch.compile(foo)

In [48]:
foo

OptimizedModule(
  (_orig_mod): Net(
    (fc): Linear(in_features=128, out_features=10, bias=True)
  )
)

When enable `inductor.debug`, it could dump the python code it codegened.

In [49]:
a = torch.randn((2, 128))

foo(a)

[2023-02-23 11:52:14,702] torch._inductor.compile_fx: [INFO] Step 3: torchinductor compiling FORWARDS graph 2
[2023-02-23 11:52:14,721] torch._inductor.compile_fx: [INFO] Step 3: torchinductor done compiling FORWARDS graph 2



from ctypes import c_void_p, c_long
import torch
import math
import random
from torch import empty_strided, as_strided, device
from torch._inductor.codecache import AsyncCompile
from torch._inductor.select_algorithm import extern_kernels

aten = torch.ops.aten
assert_size_stride = torch._C._dynamo.guards.assert_size_stride
async_compile = AsyncCompile()

import triton
import triton.language as tl
from torch._inductor.triton_ops.autotune import grid
from torch._C import _cuda_getCurrentRawStream as get_cuda_stream


kernel_cpp_0 = async_compile.cpp('''
#include "/tmp/torchinductor_root/zt/cztcl2vp5yqlnhofzpqfficjcxgyict6e3xhfdd7sdbkipp4p44x.h"
extern "C" void kernel(float* __restrict__ in_out_ptr0,
                       bool* __restrict__ out_ptr0)
{
    {
        #pragma GCC ivdep
        for(long i0=0; i0<20; i0+=1)
        {
            auto tmp0 = in_out_ptr0[i0];
            auto tmp1 = tmp0 * (tmp0>0);
            auto tmp2 = static_cast<float>(0);
            auto tmp3 = tmp1 <

/usr/local/lib/python3.8/dist-packages/torch/_functorch/aot_autograd.py:1251: UserWarning: Your compiler for AOTAutograd is returning a a function that doesn't take boxed arguments. Please wrap it with functorch.compile.make_boxed_func or handle the boxed arguments yourself. See https://github.com/pytorch/pytorch/pull/83137#issuecomment-1211320670 for rationale.
  warnings.warn(


tensor([[0.0000, 0.0000, 0.4226, 0.3017, 0.0000, 0.0000, 0.5286, 0.5539, 0.2217,
         0.0000],
        [0.0000, 0.0000, 0.8061, 0.0921, 0.0000, 0.0000, -0.0000, 0.3405, 0.4165,
         -0.0000]], grad_fn=<CompiledFunctionBackward>)

In [3]:
def my_compiler( 
        gm: torch.fx.GraphModule,
        example_inputs: List[torch.Tensor]):
    print("my_compiler() called with FX graph:")
    gm.graph.print_tabular(); print()
    #print(f"code: {gm.graph.python_code()}")
    return gm.forward  # python callable

In [ ]:
def toy_example(a, b):
    x = a / (torch.abs(a) + 1)
    if b.sum() < 0:
        b = b * -1
    return x * b

In [ ]:
torch._dynamo.reset()
toy_example = optimize(my_compiler)(toy_example)

In [ ]:
a = torch.tensor(data=[1. for i in range(6)]).reshape((2,3))
a_neg = torch.tensor(data=[-1. for i in range(6)]).reshape((2,3))
b = torch.randn((2, 3))
for i in range(4):
    print(f">>> Iteration {i}")
    toy_example(a, b * (-1) ** i)

>>> Iteration 0
my_compiler() called with FX graph:
opcode         name     target                                                  args              kwargs
-------------  -------  ------------------------------------------------------  ----------------  --------
placeholder    a        a                                                       ()                {}
placeholder    b        b                                                       ()                {}
call_function  abs_1    <built-in method abs of type object at 0x7f21751e6b80>  (a,)              {}
call_function  add      <built-in function add>                                 (abs_1, 1)        {}
call_function  truediv  <built-in function truediv>                             (a, add)          {}
call_method    sum_1    sum                                                     (b,)              {}
call_function  lt       <built-in function lt>                                  (sum_1, 0)        {}
output         output   outpu

In [ ]:
func = lambda x, y : -y if x.sum() < 0 else y

In [ ]:
torch._dynamo.reset()
func = optimize(my_compiler)(func)

## The compilation cache is reused

In [ ]:
for i in range(6):
    print(f"Iteration {i}:")
    func(a, b)
    func(a_neg, b)

Iteration 0:
my_compiler() called with FX graph:
opcode         name    target                  args        kwargs
-------------  ------  ----------------------  ----------  --------
placeholder    x       x                       ()          {}
call_method    sum_1   sum                     (x,)        {}
call_function  lt      <built-in function lt>  (sum_1, 0)  {}
output         output  output                  ((lt,),)    {}

my_compiler() called with FX graph:
opcode         name    target                   args       kwargs
-------------  ------  -----------------------  ---------  --------
placeholder    y       y                        ()         {}
call_function  neg     <built-in function neg>  (y,)       {}
output         output  output                   ((neg,),)  {}

Iteration 1:
Iteration 2:
Iteration 3:
Iteration 4:
Iteration 5:


## Non-torch function call

In [ ]:
import scipy

In [ ]:
def draw_example(a, b):
    import numpy as np
    aa = np.randn((2,3))
    sum = a + b
    return sum.numpy() + aa
    return aa

In [ ]:
torch._dynamo.reset()
torch._dynamo.config.verbose=True
func = optimize(my_compiler)(draw_example)

In [ ]:
func(a, b)

InternalTorchDynamoError: 